In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib # 모델 저장/로드용

# [Legacy] 2단계 군집화(Clustering) 및 정적 맵핑 시도
# 특이사항: 최적의 K를 찾는 튜닝 과정 없이, 직관적으로 개수를 설정함 (한계점)

# --- 1. 데이터 로드 ---
print("데이터 로드 중...")
train = pd.read_feather('1차_train.feather')
test = pd.read_feather('1차_test.feather')

# --- 2. 물리적 특성 칼럼 생성 (Train/Test 공통) ---
print("물리적 특성(포화수증기압, 일교차 등) 변수 생성 중...")

def create_physics_features(df):
    # 1. 포화 수증기압차 (Saturation Deficit)
    # (기온과 이슬점의 차이로 건조한 정도를 나타냄)
    df['saturation_deficit'] = df['temp_a'] - df['dew_point']
    
    # 2. 일교차 (Diurnal Temp Range)
    df['diurnal_temp_range'] = df['temp_max'] - df['temp_min']
    
    # 3. 태양 노출 계수 (체감온도 - 그늘체감온도)
    df['sun_exposure'] = df['real_feel_temp'] - df['real_feel_temp_shade']
    
    # 4. 안개 여부 (시정 1km 미만)
    df['is_foggy'] = (df['vis'] < 1).astype(int)
    
    return df

train = create_physics_features(train)
test = create_physics_features(test)
print("물리 변수 생성 완료.")

# --- 3. 프로필 기반 통계 추출 (Train 기준) ---
print("Train 데이터 기준으로 발전소별 기후 프로필 생성...")
# (주의: Test 데이터의 정보는 사용하지 않음 - Data Leakage 방지)

pv_id_stats = train.groupby('pv_id')[['temp_a', 'humidity', 'wind_spd_a', 'pressure', 'vis']].mean().reset_index()
pv_id_stats.rename(columns={
    'temp_a': 'avg_temp', 'humidity': 'avg_humidity', 
    'wind_spd_a': 'avg_wind', 'pressure': 'avg_pressure', 'vis': 'avg_vis'
}, inplace=True)

# Train에 병합
train_loc = train[['pv_id', 'coord1', 'coord2']].drop_duplicates(subset=['pv_id'])
train_loc = pd.merge(train_loc, pv_id_stats, on='pv_id', how='left').fillna(0)

# --- 4. 이중 군집화 학습 (Fitting) ---
print("군집화 모델 학습 시작 (K값 튜닝 없이 직관적으로 설정함)...")

# [1] 위치 군집 (Location Cluster)
# 설정 의도: 미세한 지형 차이를 반영하기 위해 80개로 잘게 쪼갬
# 한계: Elbow Method 같은 최적화 과정 없이 임의로 80개 설정
loc_features = ['coord1', 'coord2']
scaler_loc = StandardScaler()
train_scaled_loc = scaler_loc.fit_transform(train_loc[loc_features])

kmeans_loc = KMeans(n_clusters=80, random_state=42) # 튜닝 없이 80개 고정
train_loc['cluster_loc'] = kmeans_loc.fit_predict(train_scaled_loc)

# [2] 기후 군집 (Climate Cluster)
# 설정 의도: 날씨 패턴은 지역보다 넓으므로 20개로 설정
clim_features = ['avg_temp', 'avg_humidity', 'avg_wind', 'avg_pressure', 'avg_vis']
scaler_cli = StandardScaler()
train_scaled_cli = scaler_cli.fit_transform(train_loc[clim_features])

kmeans_cli = KMeans(n_clusters=20, random_state=42) # 튜닝 없이 20개 고정
train_loc['cluster_cli'] = kmeans_cli.fit_predict(train_scaled_cli)

# 모델 및 스케일러 저장 (나중에 Test 파일 생성에 쓰기 위해)
print("모델 저장 중...")
joblib.dump(scaler_loc, 'scaler_loc.pkl')
joblib.dump(kmeans_loc, 'kmeans_loc.pkl')
joblib.dump(scaler_cli, 'scaler_cli.pkl')
joblib.dump(kmeans_cli, 'kmeans_cli.pkl')

# --- 5. Test 데이터에 적용 (Transform & Predict) ---
print("저장된 모델을 불러와 Test 데이터에 군집 적용...")

# Test 데이터의 고유 위치/ID 추출
test_loc = test[['pv_id', 'coord1', 'coord2']].drop_duplicates(subset=['pv_id'])

# (1) 위치 군집 적용
# 저장해둔 Scaler와 KMeans 모델을 사용하여 Test 데이터의 군집 예측
test_scaled_loc = scaler_loc.transform(test_loc[loc_features])
test_loc['cluster_loc'] = kmeans_loc.predict(test_scaled_loc)

# (2) 기후 군집 적용
# 기후 정보는 Train에서 만든 통계(pv_id_stats)를 pv_id 기준으로 매핑(Map)
test_loc = pd.merge(test_loc, pv_id_stats, on='pv_id', how='left').fillna(0)
test_scaled_cli = scaler_cli.transform(test_loc[clim_features])
test_loc['cluster_cli'] = kmeans_cli.predict(test_scaled_cli)

# --- 6. 결과를 원본 데이터에 매핑 ---
print("군집 결과를 원본 Train/Test 데이터에 병합...")

# 맵(Map) 생성
loc_map = dict(zip(test_loc['pv_id'], test_loc['cluster_loc']))
cli_map = dict(zip(test_loc['pv_id'], test_loc['cluster_cli']))

# Train 적용
train['location_cluster'] = train['pv_id'].map(loc_map)
train['climate_cluster'] = train['pv_id'].map(cli_map)

# Test 적용
test['location_cluster'] = test['pv_id'].map(loc_map)
test['climate_cluster'] = test['pv_id'].map(cli_map)

print("완료. 군집화된 데이터셋 준비됨.")
# (이후 이 데이터로 정적 맵핑(일출몰 시간)을 시도했으나 실패함)